#Explore  the Tables

In [0]:
%sql
DESCRIBE workspace.default.dim_customers

In [0]:
%sql
SELECT * FROM workspace.default.dim_customers

#Explore  the Dates

In [0]:
%sql
SELECT 
    MIN(order_date) AS first_order_date,
    MAX(order_date) AS last_order_date,
    DATEDIFF(MONTH, MIN(order_date), MAX(order_date)) AS order_range_months
FROM workspace.default.fact_sales

#Explore the Measures

In [0]:
%sql
-- Find the Total number of Orders
SELECT COUNT(order_number) AS total_orders FROM workspace.default.fact_sales


# Explore Ranking

In [0]:
%sql
SELECT
    p.product_name,
    SUM(f.sales_amount) AS total_revenue
FROM workspace.default.fact_sales f
LEFT JOIN workspace.default.dim_products p
    ON p.product_key = f.product_key
GROUP BY p.product_name
ORDER BY total_revenue DESC
LIMIT 5;

#PySpark Analysis

In [0]:
df_customers = spark.sql("""
    SELECT
        c.customer_key,
        c.customer_number,
        CONCAT(c.first_name, ' ', c.last_name) AS customer_name,
        DATEDIFF(year, c.birthdate, CURRENT_DATE()) AS age
    FROM workspace.default.dim_customers c
""")
df_customers.display()


In [0]:
from pyspark.sql.functions import *

df_customers_grouped = (
    df_customers
        .filter(col("age") >= 18)
        .groupBy("age")
        .agg(count("*").alias("customer_count"))
        .orderBy("age")
)

df_customers_grouped.display()


In [0]:
df_customers.createOrReplaceTempView("df_customers")


In [0]:
%sql
SELECT
    CASE
        WHEN age < 18 THEN 'Under 18'
        WHEN age BETWEEN 18 AND 25 THEN '18-25'
        WHEN age BETWEEN 26 AND 35 THEN '26-35'
        WHEN age BETWEEN 36 AND 50 THEN '36-50'
        ELSE '50+'
    END AS age_group,
    COUNT(*) AS customer_count
FROM df_customers
GROUP BY
    CASE
        WHEN age < 18 THEN 'Under 18'
        WHEN age BETWEEN 18 AND 25 THEN '18-25'
        WHEN age BETWEEN 26 AND 35 THEN '26-35'
        WHEN age BETWEEN 36 AND 50 THEN '36-50'
        ELSE '50+'
    END
ORDER BY age_group;
